In [2]:
# q = 2 Maxwell and Dirac matrices along with differential forms:

# First, define the Pauli matrices:
s1 = matrix([[0, 1], [1,0]])
s2 = matrix([[0, -I], [I, 0]])
s3 = matrix([[1, 0], [0, -1]])

show('The Pauli matrices:')
show('s1: ', s1)
show('s2: ', s2)
show('s3: ', s3)

# Now define identity and the empty 2x2 matrices:
I2 = matrix([[1,0],[0,1]])
ZZ2 = matrix([[0,0],[0,0]])

# Define the Dirac matrices using tensor product:
I4 = I2.tensor_product(I2)
g0 = s3.tensor_product(I2)
g1 = I*s1.tensor_product(s1)
g2 = I*s1.tensor_product(s2)
g3 = I*s1.tensor_product(s3)
g5 = I*g0*g1*g2*g3

# Now take a look at them:
show('The Dirac matrices:')
show('g0: ', g0)
show('g1: ', g1)
show('g2: ', g2)
show('g3: ', g3)
show('g5: ', g5)

# Now define p-slash, call it P:
var('E p1 p2 p3')
P = E*g0 + p1*g1 + p2*g2 + p3*g3

# Now check it works:
show('P^2: ', expand(P^2))
# show('g5^2: ', expand(g5^2))

# Variables for custom_latex() function:
var('d0 d1 d2 d3')
var('A0 A1 A2 A3')
var('E1 E2 E3')
var('B1 B2 B3')
var('rho J1 J2 J3')

# The order of the terms in this list, is the order they will be sorted to in expressions:
custom_latex_sort_order = [-1]
custom_latex_sort_order += [d0, d1, d2, d3]
custom_latex_sort_order += [A0, A1, A2, A3]
custom_latex_sort_order += [E1, E2, E3]
custom_latex_sort_order += [B1, B2, B3]
# custom_latex_sort_order += [B1^2, B2^2, B3^2]
custom_latex_sort_order += [rho, J1, J2, J3]

# Maps indecies to their associated latex:
custom_latex_string = ['-']
custom_latex_string += [r'\partial_0', r'\partial_1', r'\partial_2', r'\partial_3']
custom_latex_string += ['A_0', 'A_1', 'A_2', 'A_3']
custom_latex_string += ['E_1', 'E_2', 'E_3']
custom_latex_string += ['B_1', 'B_2', 'B_3']
# custom_latex_string += ['B_1^2', 'B_2^2', 'B_3^2']
custom_latex_string += [r'\rho', 'J_1', 'J_2', 'J_3']


# Variables we need to help walk the parse tree:
var('sym1 sym2')

# The function that maps expression to our custom latex:
def first_custom_latex(r):
    r = expand(r)
    latex_result = ''
    if r.operator() == (sym1 * sym2).operator():
        idx_list = []
        for t in r.operands():
            idx = custom_latex_sort_order.index(t)
            idx_list.append(idx)
        idx_list.sort()
        latex_result += ' '.join(custom_latex_string[i] for i in idx_list)
    elif r.operator() == (sym1 + sym2).operator():
        latex_result += ' + '.join(custom_latex(term) for term in r.operands())
    elif r.operator() == (sym1^sym2).operator():
        head, tail = r.operands()
        head_idx = custom_latex_sort_order.index(head)
        latex_result += '%s^{%s}' % (custom_latex_string[head_idx], latex(tail))
    else:
        idx = custom_latex_sort_order.index(r)
        latex_result += custom_latex_string[idx]
    return latex_result.replace(' + - ', ' - ')

# Improved version:
def custom_latex(r):
    r = expand(r)
    latex_result = ''
    if r.operator() == (sym1 * sym2).operator():
        # show('r.operands:', r.operands())
        match = False
        for x in r.operands():
            if x < 0:
                match = True
        if match:
            # show('Match!')
            latex_result += ' - ' + custom_latex(-r)
        else:
            idx_list = []
            power_term_tails = {}
            number = 1
            for t in r.operands():
                # show('t:', t)
                if t.operator() == (sym1^sym2).operator():
                    head, tail = t.operands()
                    idx = custom_latex_sort_order.index(head)
                    idx_list.append(idx)
                    power_term_tails[idx] = tail
                else:
                    if t not in custom_latex_sort_order:
                        number = t
                    else:
                        idx = custom_latex_sort_order.index(t)
                        idx_list.append(idx)
            idx_list.sort()
            result_list = []
            if number != 1:
                result_list.append(str(number))
            for i in idx_list:
                if i in power_term_tails:
                    result_list.append(custom_latex_string[i] + '^{' + latex(power_term_tails[i]) + '}')
                else:
                    result_list.append(custom_latex_string[i])
            latex_result += ' '.join(x for x in result_list)
    elif r.operator() == (sym1 + sym2).operator():
        latex_result += ' + '.join(custom_latex(term) for term in r.operands())
    elif r.operator() == (sym1^sym2).operator():
        head, tail = r.operands()
        head_idx = custom_latex_sort_order.index(head)
        latex_result += '%s^{%s}' % (custom_latex_string[head_idx], latex(tail))
    else:
        idx = custom_latex_sort_order.index(r)
        latex_result += custom_latex_string[idx]
    # return latex_result
    return latex_result.replace(' +  - ', ' - ')

    
basis_matrices = [I2, s1, s2, s3, I4, g0, g1, g2, g3, g5]
basis_labels = ['I2', 's1', 's2', 's3', 'I4', 'g0', 'g1', 'g2', 'g3', 'g5']

# Given a matrix, extract out the component matrices:
# Handles both Pauli and Dirac matrices
def get_matrix_basis(A):
    for idx in range(len(basis_matrices)):
        matrix = basis_matrices[idx]
        if A.dimensions() != matrix.dimensions():
            continue
        n = A.nrows()  # Assume A is a square matrix
        term = (expand(A*matrix^3)).trace()/n
        if term != 0:
            label = basis_labels[idx]
            show(label, '::', term)

# Quick test:
show('Quick test of get_matrix_basis(A):')
show('get_matrix_basis(P):')
get_matrix_basis(P)

show('get_matrix_basis(P^2):')
get_matrix_basis(P^2)

# Define our differential form basis matrices:
D0 = g0
D1 = g1
D2 = g2
D3 = g3
D01 = g0*g1
D02 = g0*g2
D03 = g0*g3
D23 = g2*g3
# D13 = g1*g3
D31 = g3*g1
D12 = g1*g2
D023 = g0*g2*g3
# D013 = g0*g1*g3
D031 = g0*g3*g1
D012 = g0*g1*g2
D123 = g1*g2*g3
D0123 = g0*g1*g2*g3

# Define our matrices with respect to k:
d_basis_matrices = {}
d_basis_labels = {}

d_basis_matrices[0] = [I2]
d_basis_labels[0] = ['I2']

d_basis_matrices[1] = [D0, D1, D2, D3]
d_basis_labels[1] = ['D0', 'D1', 'D2', 'D3']

# d_basis_matrices[2] = [D01, D02, D03, D23, D13, D12]
# d_basis_labels[2] = ['D01', 'D02', 'D03', 'D23', 'D13', 'D12']

d_basis_matrices[2] = [D01, D02, D03, D23, D31, D12]
d_basis_labels[2] = ['D01', 'D02', 'D03', 'D23', 'D31', 'D12']

# d_basis_matrices[3] = [D023, D013, D012, D123]
# d_basis_labels[3] = ['D023', 'D013', 'D012', 'D123']

d_basis_matrices[3] = [D023, D031, D012, D123]
d_basis_labels[3] = ['D023', 'D031', 'D012', 'D123']

d_basis_matrices[4] = [D0123]
d_basis_labels[4] = ['D0123']

# Now our extract basis function:
def get_d_basis(k, A, use_latex=False):
    if k not in d_basis_matrices:
        return
    latex_result = ""
    matrices = d_basis_matrices[k]
    labels = d_basis_labels[k]
    for idx in range(len(matrices)):
        matrix = matrices[idx]
        if A.dimensions() != matrix.dimensions():
            continue
        n = A.nrows()  # Assume A is a square matrix
        term = (expand(A*matrix^3)).trace()/n
        if term != 0:
            label = labels[idx]
            if not use_latex:
                show(label, '::', term)
            else:
                # sterm = label + ': ' + str(term)
                # latex(sterm)
                latex_result += '& ' + latex(label + ': ') + custom_latex(term) + "\\\\\n"
    if not use_latex:
        return
    print(latex_result)
    return latex_result
                
 
# Now define our exterior derivative for q = 2, dim = 4:
var('d0 d1 d2 d3')
d = d0*D0 + d1*D1 + d2*D2 + d3*D3

# Now define a couple of 1-forms:
var('A0 A1 A2 A3')
var('B0 B1 B2 B3')
A = A0*D0 + A1*D1 + A2*D2 + A3*D3
B = B0*D0 + B1*D1 + B2*D2 + B3*D3

# Now define Maxwell's F:
var('E1 E2 E3 B1 B2 B3')
F = - E1*D01 - E2*D02 - E3*D03 + B1*D23 + B2*D31 + B3*D12

# Now a quick test:
show('Quick test of get_d_basis(k, A):')
show('The P matrix:')
get_d_basis(1, P)

show('The F matrix:')
get_d_basis(2, F)

# Now dF:
show('--------------------')
show('dF as a 3-form:')
get_d_basis(3, d*F)

show('--------------------')
show('dF as matrix basis:')
get_matrix_basis(d*F)

show('--------------------')
show('dF as a 1-form:')
get_d_basis(1, d*F)


# define the Hodge star operator:
# NB: Only works for 1-forms:
def star(A):
    r = 0
    for idx in range(len(basis_matrices)):
        matrix = basis_matrices[idx]
        if A.dimensions() != matrix.dimensions():
            continue
        n = A.nrows()  # Assume A is a square matrix
        term = (expand(A*matrix^3)).trace()/n
        if term != 0:
            r += term*matrix*D0123
    return r

# Now test the Hodge star operator:
show('-------------------------')
show('Test of the Hodge star operator:')
show('P wedge star P:')
get_d_basis(4, P*star(P))

# show('-------------------------')
# get_d_basis(4, F*star(F))

# Improved Hodge star operator:
def star_k(k, A):
    if k not in d_basis_matrices:
        return
    matrices = d_basis_matrices[k]
    labels = d_basis_labels[k]
    r = 0
    for idx in range(len(matrices)):
        matrix = matrices[idx]
        if A.dimensions() != matrix.dimensions():
            continue
        n = A.nrows()  # Assume A is a square matrix
        term = (expand(A*matrix^3)).trace()/n
        if term != 0:
            r += term*matrix*D0123
    return r

show('-------------------------')
show('F wedge F, as a 4-form:')
get_d_basis(4, F*star_k(2, F))

# show('----------------------')
# show('dA as a 2-form:')
# get_d_basis(2, d*A)


# Now define the vector current:
show('----------------------------------------')
var('rho J1 J2 J3')
J = - rho*D0 + J1*D1 + J2*D2 + J3*D3

show('d star J, as a 4-form:')
get_d_basis(4, d*star(J))


# Now quick test of the Pauli identity:
show('--------------------------------------------------')
var('a1 a2 a3')
var('b1 b2 b3')

a = a1*s1 + a2*s2 + a3*s3
b = b1*s1 + b2*s2 + b3*s3
show('Test the Pauli identity:')
show('(a.s)*(b.s):')
get_matrix_basis(a*b)


# Now a quick test of Hodge star dual for two 1-forms:
show('-------------------')
show('Test of Hodge star for 1-forms:')
show('A wedge star B, as a 4-form:')
get_d_basis(4, A*star(B))

show('--------------------------')
show('Ditto, this time using get_matrix_basis:')
get_matrix_basis(A*star(B))

'The Pauli matrices:'

's1: ' [0 1]
[1 0]

's2: ' [ 0 -I]
[ I  0]

's3: ' [ 1  0]
[ 0 -1]

'The Dirac matrices:'

...

'A wedge star B, as a 4-form:'

'D0123' '::' A0*B0 - A1*B1 - A2*B2 - A3*B3

'--------------------------'

'Ditto, this time using get_matrix_basis:'

'g5' '::' -I*A0*B0 + I*A1*B1 + I*A2*B2 + I*A3*B3

In [4]:
# In this section we define the dim-table function
# Ie, the dimensions of k-forms for a given d and q
import itertools
# from collections import Counter

def dim_table(d, q):
    basis_indices = []
    if d == 3 and q == 2:
        basis_indices = [1,2,3]
    if d == 4 and q == 2:
        basis_indices = [0,1,2,3]
    if d == 3 and q == 3:
        basis_indices = [1,2,3]
    if d == 4 and q == 3:
        basis_indices = [0,1,2,3,4,5,6]
    basis_dict = {}
    k = 0
    while True:
        basis_dict[k] = []
        basis_count = 0
        for elt in itertools.combinations_with_replacement(basis_indices, k):
            match = False
            for b in basis_indices:
                if elt.count(b) >= q:
                    match = True
                    break
            if match:
                continue
            # print(elt)
            basis_dict[k] += [elt]
            basis_count += 1
        k += 1
        if basis_count == 0:
            break
    rows = [['k', 'dim', 'basis']]
    for i in range(k - 1):
        basis_result = basis_dict[i]
        str_basis_result = " ".join("".join(str(x) for x in b) for b in basis_result)
        rows += [[i, len(basis_result), str_basis_result]]
    show(table(rows, header_row=True))
    print(latex(table(rows, header_row=True, frame=False)))
    
    # return basis_dict

show('The 4 dimension space-time, q = 2, k-form dimension table:')
dim_table(4, 2)

'The 4 dimension space-time, q = 2, k-form dimension table:'

k,dim,basis
,,
,,0 1 2 3
,,01 02 03 12 13 23
,,012 013 023 123
,,0123


\begin{tabular}{lll}
k & dim & basis \\ \hline
$0$ & $1$ &  \\
$1$ & $4$ & 0 1 2 3 \\
$2$ & $6$ & 01 02 03 12 13 23 \\
$3$ & $4$ & 012 013 023 123 \\
$4$ & $1$ & 0123 \\
\end{tabular}


In [6]:
# A final summary of Maxwell's equations:
show('Summary of Maxwells equations:')
# Define the F matrix:
F = - E1*D01 - E2*D02 - E3*D03 + B1*D23 + B2*D31 + B3*D12

show('F as a 2-form:')
get_d_basis(2, F)

show('F wedge star F, as a 4-form:')
get_d_basis(4, F*star_k(2, F))

show('----------------------')
show('Now as latex:')
show('dF, 1-form:')
get_d_basis(1, d*F, True)
show('----------------------')
show('dF, 3-form:')
get_d_basis(3, d*F, True)

show('----------------------')
show('dA, 2-form:')
get_d_basis(2, d*A, True)

show('F wedge *F, 4-form:')
get_d_basis(4, F*star_k(2, F), True)

show('d wedge *J, 4-form:')
get_d_basis(4, d*star_k(1, J), True)

'Summary of Maxwells equations:'

'F as a 2-form:'

'D01' '::' -E1

'D02' '::' -E2

'D03' '::' -E3

...

'F wedge *F, 4-form:'

& \text{\texttt{D0123:{ }}} - B_1^{2} - B_2^{2} - B_3^{2} + E_1^{2} + E_2^{2} + E_3^{2} \\



'd wedge *J, 4-form:'

& \text{\texttt{D0123:{ }}} - \partial_1 J_1 - \partial_2 J_2 - \partial_3 J_3 - \partial_0 \rho \\



& \text{\texttt{D0123:{ }}} - \partial_1 J_1 - \partial_2 J_2 - \partial_3 J_3 - \partial_0 \rho \\
